<a href="https://colab.research.google.com/github/sanderanjos/Unidade-1-sessao-1_2/blob/main/Servidor_cliente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nova seção

In [ ]:
import socket
import threading
import time

# Dados do servidor
HOST = '127.0.0.1'  # Loopback IP, significa 'este computador'
PORT = 65432        # Porta arbitrária para comunicação (deve ser maior que 1023)

def handle_client(conn, addr):
    """Função para lidar com a conexão de um cliente específico."""
    print(f"[{time.strftime('%H:%M:%S')}] Conectado por {addr}")
    try:
        # Recebe dados do cliente
        data = conn.recv(1024) # buffer de 1024 bytes
        if not data:
            print(f"[{time.strftime('%H:%M:%S')}] Cliente {addr} desconectou.")
            return

        message = data.decode('utf-8')
        print(f"[{time.strftime('%H:%M:%S')}] Recebido de {addr}: '{message}'")

        # Processa a mensagem (ex: converte para maiúsculas)
        response = f"SERVIDOR: Sua mensagem '{message}' foi processada para '{message.upper()}'."
        conn.sendall(response.encode('utf-8')) # Envia a resposta de volta ao cliente
        print(f"[{time.strftime('%H:%M:%S')}] Enviado para {addr}: '{response}'")

    except Exception as e:
        print(f"[{time.strftime('%H:%M:%S')}] Erro na comunicação com {addr}: {e}")
    finally:
        conn.close() # Garante que a conexão seja fechada
        print(f"[{time.strftime('%H:%M:%S')}] Conexão com {addr} fechada.")

def start_server():
    """Inicia o servidor e aguarda por conexões."""
    # Cria um socket TCP/IP
    # AF_INET indica IPv4, SOCK_STREAM indica TCP
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT)) # Associa o socket ao endereço e porta
        s.listen() # Começa a escutar por conexões (pode aceitar 1 conexão na fila)
        print(f"[{time.strftime('%H:%M:%S')}] Servidor escutando em {HOST}:{PORT}...")

        while True:
            # Aceita uma nova conexão
            conn, addr = s.accept() # Bloqueia até que um cliente se conecte
            # Cria uma nova thread para lidar com o cliente, permitindo múltiplos clientes (conceitualmente)
            client_thread = threading.Thread(target=handle_client, args=(conn, addr))
            client_thread.start()

# Inicia o servidor em uma thread separada para não bloquear o ambiente do Colab
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True # Permite que a thread do servidor seja encerrada com o programa principal
server_thread.start()
print("Servidor iniciado em segundo plano. Aguardando a execução do cliente...")

# Para garantir que o servidor tenha tempo de inicializar antes de você tentar conectar o cliente
time.sleep(2)

[00:47:18] Servidor escutando em 127.0.0.1:65432...
Servidor iniciado em segundo plano. Aguardando a execução do cliente...


# Nova seção

In [ ]:
import socket
import time

# Dados do servidor (devem ser os mesmos que o servidor está escutando)
HOST = '127.0.0.1'
PORT = 65432

def start_client():
    """Inicia o cliente, conecta-se ao servidor e envia uma mensagem."""
    # Cria um socket TCP/IP
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            s.connect((HOST, PORT)) # Tenta conectar ao servidor
            print(f"[{time.strftime('%H:%M:%S')}] Conectado ao servidor em {HOST}:{PORT}")

            message = "Olá, Servidor! Esta é uma mensagem do cliente."
            s.sendall(message.encode('utf-8')) # Envia a mensagem codificada em bytes
            print(f"[{time.strftime('%H:%M:%S')}] Enviado: '{message}'")

            data = s.recv(1024) # Recebe a resposta do servidor
            response = data.decode('utf-8')
            print(f"[{time.strftime('%H:%M:%S')}] Recebido do servidor: '{response}'")

        except ConnectionRefusedError:
            print(f"[{time.strftime('%H:%M:%S')}] Erro: Conexão recusada. Certifique-se de que o servidor está rodando em {HOST}:{PORT}")
        except Exception as e:
            print(f"[{time.strftime('%H:%M:%S')}] Erro no cliente: {e}")
        finally:
            s.close() # Garante que a conexão seja fechada

# Executa o cliente
start_client()

[00:47:20] Conectado ao servidor em 127.0.0.1:65432
[00:47:20] Enviado: 'Olá, Servidor! Esta é uma mensagem do cliente.'
[00:47:20] Conectado por ('127.0.0.1', 59402)
[00:47:20] Recebido de ('127.0.0.1', 59402): 'Olá, Servidor! Esta é uma mensagem do cliente.'
[00:47:20] Recebido do servidor: 'SERVIDOR: Sua mensagem 'Olá, Servidor! Esta é uma mensagem do cliente.' foi processada para 'OLÁ, SERVIDOR! ESTA É UMA MENSAGEM DO CLIENTE.'.'
[00:47:20] Enviado para ('127.0.0.1', 59402): 'SERVIDOR: Sua mensagem 'Olá, Servidor! Esta é uma mensagem do cliente.' foi processada para 'OLÁ, SERVIDOR! ESTA É UMA MENSAGEM DO CLIENTE.'.'
[00:47:20] Conexão com ('127.0.0.1', 59402) fechada.
